# Conversation With History

Hướng dẫn này bao gồm cách tạo một `Chain` đa lượt (multi-turn) có khả năng ghi nhớ các cuộc trò chuyện trước đó, sử dụng LangChain.<br>
Nó bao gồm việc quản lý lịch sử cuộc trò chuyện (conversation history), định nghĩa một `ChatPromptTemplate`, và sử dụng LLM để tạo chain.<br>
Lịch sử cuộc trò chuyện được quản lý bằng cách sử dụng `chat_history`.


In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv(override=True, dotenv_path="../.env")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

## Creating a Chain that Remembers Previous Conversations

`MessagesPlaceholder` là một lớp (class) trong LangChain được sử dụng để xử lý lịch sử cuộc trò chuyện (conversation history). Nó chủ yếu được sử dụng trong chatbot hoặc các hệ thống hội thoại đa lượt (multi-turn) để lưu trữ và tái sử dụng nội dung hội thoại trước đó.

### Vai trò chính

**Chèn Lịch Sử Cuộc Trò Chuyện (Inserting Conversation History)**:

* Được sử dụng để chèn các cuộc hội thoại trước đó (ví dụ: lịch sử hỏi đáp) vào prompt.
* Điều này cho phép model hiểu ngữ cảnh của cuộc trò chuyện và tạo ra các phản hồi phù hợp.

**Quản Lý Biến (Managing Variables)**:

* Quản lý lịch sử cuộc trò chuyện trong prompt bằng cách sử dụng một key cụ thể (ví dụ: `chat_history`).
* Nó được liên kết với một tên biến do người dùng định nghĩa.

### Cách Sử Dụng

`MessagesPlaceholder(variable_name="chat_history")`

* Ở đây, `chat_history` là tên biến nơi lịch sử cuộc trò chuyện được lưu trữ.
* Khi cuộc trò chuyện tiến triển, `chat_history` liên tục được cập nhật với các cặp câu hỏi và câu trả lời.


In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser

# Define the prompt.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a Question-Answering chatbot. Please provide answers to the given questions.",
        ),
        # Use "chat_history" as the key for conversation history without modifying it if possible.
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "#Question:\n{question}"),  # Use user input as a variable.
    ]
)


# Create a basic chain.
chain = prompt | llm | StrOutputParser()

## Creating a Chain to Record Conversations (`chain_with_history`)

Trong bước này, chúng ta tạo một hệ thống quản lý **lịch sử cuộc trò chuyện dựa trên phiên (session-based conversation history)** và tạo ra một **chuỗi thực thi (executable chain)**.

-   **Quản Lý Lịch Sử Cuộc Trò Chuyện (Conversation History Management)**: Từ điển `store` lưu và truy xuất lịch sử cuộc trò chuyện (`ChatMessageHistory`) theo ID phiên (session ID). Nếu một phiên không tồn tại, một phiên mới sẽ được tạo.
-   **Thực Thi Chuỗi (Chain Execution)**: `RunnableWithMessageHistory` kết hợp lịch sử cuộc trò chuyện và chuỗi để tạo ra phản hồi dựa trên câu hỏi của người dùng và lịch sử cuộc trò chuyện. Cấu trúc này được thiết kế để quản lý hiệu quả các cuộc trò chuyện đa lượt (multi-turn conversations).


In [3]:
# A dictionary to store session history.
store = {}

# A function to retrieve session history based on the session ID.
def get_session_history(session_ids):
    print(f"[Conversation session ID]: {session_ids}")
    if session_ids not in store:  # When the session ID is not in the store.
        # Create a new ChatMessageHistory object and save it in the store.
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # Return the session history for the given session ID.

In [4]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # A function to retrieve session history.
    input_messages_key="question",  # The key where the user's question will be inserted into the template variable.
    history_messages_key="chat_history",  # The key for the message in the history.
)

In [5]:
# Execute the first question.
chain_with_history.invoke(
    # Question input.
    {"question": "My name is Teddy."},
    # Record conversations based on the session ID.
    config={"configurable": {"session_id": "abc123"}},
)

[Conversation session ID]: abc123


'Nice to meet you, Teddy! How can I assist you today?'

In [6]:
chain_with_history.invoke(
    # Question input.
    {"question": "What's my name?"},
    # Record conversations based on the session ID.
    config={"configurable": {"session_id": "abc123"}},
)

[Conversation session ID]: abc123


'Your name is Teddy.'

In [7]:
chain_with_history.invoke(
    # Question input.
    {"question": "What's my name?"},
    # Record conversations based on the session ID.
    config={"configurable": {"session_id": "abc1234"}},
)

[Conversation session ID]: abc1234


"I'm sorry, but I don't have access to personal information such as your name. How can I assist you today?"